### Packages

In [1]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Functions

In [50]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

function Y(x)
    a = 0.1836121365412579
    b = 0.2749462902187088
    c = 0.25
    return  a * (x.^2) + b * x .+ c 
end

function print_output(index, I0, U0, initial_condition)
    println("index: $index");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("initial condition: $initial_condition")
end
function separate()
    println("_________________")
    println("")
end

separate (generic function with 1 method)

### Parametres

In [45]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5
parametres = SA[ τ, α, τ_D, J, NaN, ΔU0, τ_y, β, xthr, ythr, NaN ]
timefinal = 500.0
println(parametres)

[0.013, 1.5, 0.15, 3.07, NaN, 0.305, 1.8, 0.4375, 0.9, 0.5, NaN]


### Grid

In [53]:
startI = -1.8
endI = -1.24798
lengthI = 100

rangeI = range( startI, endI, length = lengthI )
rangeU0 = Y( rangeI );

In [54]:
matrix_spectrum = zeros( length( rangeI ), length( rangeI ), 4 )
matrix_initial_condition = zeros( length( rangeI ), length( rangeI ), 4 );
file = "output_from_function.txt"

"output_from_function.txt"

In [55]:
for (index, I0) in enumerate(rangeI)
    """
        Итерируемся по точкам c функции
    """
    if index == 1
        global initial_condition_function = SA[0.0, 0.2, 0.1, 0.1]
    end
    
    U0 = Y(I0)
    
    print_output(index, I0, U0, initial_condition_function)
    
    parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
    ds = ContinuousDynamicalSystem( model, initial_condition_function, parametres )
    
    tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
    
    initial_condition_function = tr[end, :]
    println("last point: $initial_condition_function");flush(stdout)
    λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
    println("LSE: $λλ");flush(stdout)
    """
        this save output
    """
    open(file, "a") do io
        println(io, " $I0 $U0 $(λλ[1])")
    end
end

index: 1
I0: -1.8
U0: 0.34999999999999987
initial condition: [0.0, 0.2, 0.1, 0.1]
last point: [1.9774203443433476, 0.8191410397203346, 0.7440336651009856, 0.039499383434498825]
LSE: [-0.00022855632034940893, -0.555555539508975, -2.278804953508714, -8.775049309549207]
index: 2
I0: -1.7944240404040404
U0: 0.347853068273151
initial condition: [1.9774203443433476, 0.8191410397203346, 0.7440336651009856, 0.039499383434498825]
last point: [5.506264376920801, 0.38510410385210425, 0.845760894789485, 0.03837810268806744]
LSE: [-0.0068565563526544054, -0.5555555678114624, -2.262877785237668, -8.697333338302764]
index: 3
I0: -1.7888480808080809
U0: 0.3457175540356773
initial condition: [5.506264376920801, 0.38510410385210425, 0.845760894789485, 0.03837810268806744]
last point: [1.0037439326501696, 0.7519720011578845, 0.777257011795642, 0.035057416226887766]
LSE: [0.009975960197391717, -0.5555554847239966, -2.249623243323895, -8.620089373721122]
index: 4
I0: -1.7832721212121212
U0: 0.3435934572875

initial condition: [1.6204702670015878, 0.7995860564060754, 0.7230056986865923, 0.030169337717588094]
last point: [9.35057686291937, 0.3634567696523331, 0.8118986688711326, 0.03114536622532515]
LSE: [-0.0015536913628586956, -0.5555555633496897, -1.9039889173000588, -6.7759372572634495]
index: 26
I0: -1.66060101010101
U0: 0.29975195364127505
initial condition: [9.35057686291937, 0.3634567696523331, 0.8118986688711326, 0.03114536622532515]
last point: [3.3311251790252543, 0.46838008695480443, 0.8012793149951187, 0.030020247818441263]
LSE: [0.0011271823821440484, -0.5555555562553219, -1.8814678585188436, -6.689796094363063]
index: 27
I0: -1.6550250505050506
U0: 0.29789045914880086
initial condition: [3.3311251790252543, 0.46838008695480443, 0.8012793149951187, 0.030020247818441263]
last point: [11.706671706040169, 0.718155184008761, 0.7115497147811315, 0.03249025130302712]
LSE: [-0.008472930550391708, -0.5555555610941942, -1.8635579027977136, -6.600583072732624]
index: 28
I0: -1.649449090

U0: 0.25982620512623694
initial condition: [1.7206220401337624, 0.6050420317031745, 0.7427458367133846, 0.02480768294897542]
last point: [2.4340959413856478, 0.5488648341230751, 0.7512010494441136, 0.025033520652727296]
LSE: [0.006317856584458395, -0.5555555543826637, -1.426381100117866, -4.733539046579552]
index: 50
I0: -1.5267779797979797
U0: 0.2582273128893873
initial condition: [2.4340959413856478, 0.5488648341230751, 0.7512010494441136, 0.025033520652727296]
last point: [1.5078201461459073, 0.6431537070446192, 0.7313381331222032, 0.02424936466983475]
LSE: [-0.0006390382430088279, -0.555555556471199, -1.4057829327254308, -4.645105057317079]
index: 51
I0: -1.5212020202020202
U0: 0.2566398381419125
initial condition: [1.5078201461459073, 0.6431537070446192, 0.7313381331222032, 0.02424936466983475]
last point: [1.4265161279017553, 0.7429206765657056, 0.7025403247314633, 0.023755861980625848]
LSE: [0.008053338116879999, -0.5555555544027131, -1.3839843963901424, -4.564567299911122]
inde

index: 73
I0: -1.398530909090909
U0: 0.22460401850933792
initial condition: [2.1828563740258065, 0.6263988625372005, 0.694807198280715, 0.02140869076863944]
last point: [2.543383872814475, 0.786251652997082, 0.6394599400119251, 0.02251625875241148]
LSE: [-0.0013794350394853046, -0.5555555590312603, -0.8010131140329703, -2.778665879505282]
index: 74
I0: -1.3929549494949496
U0: 0.22327914601748772
initial condition: [2.543383872814475, 0.786251652997082, 0.6394599400119251, 0.02251625875241148]
last point: [1.9403736315160773, 0.7645887693424331, 0.6513310534646202, 0.021115038403242553]
LSE: [0.004345728001165597, -0.5555555570965952, -0.7698344295356826, -2.7032762265218917]
index: 75
I0: -1.38737898989899
U0: 0.22196569101501246
initial condition: [1.9403736315160773, 0.7645887693424331, 0.6513310534646202, 0.021115038403242553]
last point: [18.105186526218617, 0.5242697289852437, 0.6738477499294411, 0.023376922206637857]
LSE: [-0.0073833203381700125, -0.5555555616643937, -0.737302138

index: 97
I0: -1.2647078787878787
U0: 0.19595830577242707
initial condition: [7.974041884996481, 0.5486837511595972, 0.6876842837051201, 0.0006988288602759761]
last point: [7.947356923482322, 0.5502665118459157, 0.6855958292103942, 0.0007212837071456986]
LSE: [-0.5555555555756403, -0.6574601003144572, -0.6651585971221649, -1.6271426497996937]
index: 98
I0: -1.2591319191919192
U0: 0.19490745302557633
initial condition: [7.947356923482322, 0.5502665118459157, 0.6855958292103942, 0.0007212837071456986]
last point: [7.920250401336667, 0.5518700925086937, 0.6834974502659618, 0.0007447691568232384]
LSE: [-0.5555555555761319, -0.6663357452383571, -0.6556564919623348, -1.600828109114958]
index: 99
I0: -1.2535559595959596
U0: 0.19386801776810064
initial condition: [7.920250401336667, 0.5518700925086937, 0.6834974502659618, 0.0007447691568232384]
last point: [7.892711272585649, 0.5534950435871172, 0.6813889288262498, 0.0007693469940260485]
LSE: [-0.5555555555766737, -0.6600026260062543, -0.66114

In [56]:
rangeI.step

Base.TwicePrecision{Float64}(0.005575959595959468, 1.2829674472073987e-16)